In [1]:
import tensorflow as tf
import os
import collections
import numpy as np

# TODO(zongheng): print some prediction from test set

dropout_keepprob = None

# With gaussian weights.
# 500 training pts; 1000 testing.
PATH = "5rel-10numAttrs-22totalAttrs-100maxTblSize-10buckets-3000.csv"
NUM_FIELDS = 34
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [64,64]; lr =  1e-3
batch_size = 500
num_steps = 5500

# With 5 random op types.
PATH = "5rel-10numAttrs-28totalAttrs-100maxTblSize-10buckets-3000-1526330982.csv"
NUM_FIELDS = 40
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [128,128]; lr =  5e-3
batch_size = 1024

# 5 random op types.  Larger maxTblSize & buckets. 
PATH = "5rel-10numAttrs-24totalAttrs-1000maxTblSize-100buckets-3000-1526336271.csv"
NUM_FIELDS = 36
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [256,256]; lr =  3e-3
lr_bounds = [10000,]; lr_vals = [1e-3, 5e-4]
batch_size = 2000
dropout_keepprob = 0.9

/Users/zongheng/anaconda/envs/ray-0321/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


1526336271
no whitening, net = [128,128]; lr =  3e-3; batch_size = 512; dropout_keepprob = 0.9
![sup](./1.png)
lr_bounds = [2000, 6000,]; lr_vals = [3e-3, 1e-3, 5e-4]
![inf](./2.png)

In [2]:
EPS = 1e-8
def DatasetFromPath(pathname):
    dataset = tf.data.TextLineDataset(pathname)

    def ParseLine(line):
        fields = tf.decode_csv(line, [[0.0]] * NUM_FIELDS)
        # Last number is label.
        return tf.reshape(tf.stack(fields[:-1]), [-1]), fields[-1]

    def MeanCenterBatch(feat, lab):
        feat_mean, feat_var = tf.nn.moments(feat, axes=[0])
        lab_mean, lab_var = tf.nn.moments(lab, axes=[0])
        feat_std = tf.sqrt(feat_var)
        lab_std = tf.sqrt(lab_var)
        feat = (feat - feat_mean) / (feat_std + EPS)
        lab = (lab - lab_mean) / (lab_std + EPS)
        return feat, lab, feat_mean, feat_std, lab_mean, lab_std
    
#     # K-point dataset, for sanity checking.
#     K = 10
#     dataset = dataset.take(K).map(ParseLine)
#     dataset = dataset.shuffle(20000, reshuffle_each_iteration=False) \
#         .batch(K).map(MeanCenterBatch) \
#         .repeat()

    dataset = dataset.map(ParseLine).cache()
    dataset = dataset.shuffle(20000, reshuffle_each_iteration=True) \
        .repeat() \
        .batch(batch_size) \
        .map(MeanCenterBatch)

    print(dataset.output_types)
    print(dataset.output_shapes)
    return dataset

# Train.
dataset = DatasetFromPath(TRAIN_PATH)
iterator = tf.data.Iterator.from_structure(
    dataset.output_types, dataset.output_shapes)
iterator_init = iterator.make_initializer(dataset)
# Test.
test_dataset = DatasetFromPath(TEST_PATH)
test_iterator_init = iterator.make_initializer(test_dataset)
# Shared.
tup = iterator.get_next()
if len(tup) == 2:
    feature_batch, label_batch = iterator.get_next()
    feat_mean, feat_std, lab_mean, lab_std = (None,) * 4
elif len(tup) == 6:
    feature_batch, label_batch, feat_mean, feat_std, lab_mean, lab_std = iterator.get_next()
else:
    assert False

# mean, var = tf.nn.moments(feature_batch, axes=[0])
# feature_batch = (feature_batch - mean) / tf.sqrt(var)
# mean, var = tf.nn.moments(label_batch, axes=[0])
# label_batch = (label_batch - mean) / var

(tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(35)]), TensorShape([Dimension(None)]), TensorShape([Dimension(35)]), TensorShape([Dimension(35)]), TensorShape([]), TensorShape([]))
(tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(35)]), TensorShape([Dimension(None)]), TensorShape([Dimension(35)]), TensorShape([Dimension(35)]), TensorShape([]), TensorShape([]))


In [3]:
def manual_stepping(global_step, boundaries, rates):
  """Manually stepped learning rate schedule.

  This function provides fine grained control over learning rates.  One must
  specify a sequence of learning rates as well as a set of integer steps
  at which the current learning rate must transition to the next.  For example,
  if boundaries = [5, 10] and rates = [.1, .01, .001], then the learning
  rate returned by this function is .1 for global_step=0,...,4, .01 for
  global_step=5...9, and .001 for global_step=10 and onward.

  Args:
    global_step: int64 (scalar) tensor representing global step.
    boundaries: a list of global steps at which to switch learning
      rates.  This list is assumed to consist of increasing positive integers.
    rates: a list of (float) learning rates corresponding to intervals between
      the boundaries.  The length of this list must be exactly
      len(boundaries) + 1.

  Returns:
    a (scalar) float tensor representing learning rate
  Raises:
    ValueError: if one of the following checks fails:
      1. boundaries is a strictly increasing list of positive integers
      2. len(rates) == len(boundaries) + 1
  """
  if any([b < 0 for b in boundaries]) or any(
      [not isinstance(b, int) for b in boundaries]):
    raise ValueError('boundaries must be a list of positive integers')
  if any([bnext <= b for bnext, b in zip(boundaries[1:], boundaries[:-1])]):
    raise ValueError('Entries in boundaries must be strictly increasing.')
  if any([not isinstance(r, float) for r in rates]):
    raise ValueError('Learning rates must be floats')
  if len(rates) != len(boundaries) + 1:
    raise ValueError('Number of provided learning rates must exceed '
                     'number of boundary points by exactly 1.')
  step_boundaries = tf.constant(boundaries, tf.int64)
  learning_rates = tf.constant(rates, tf.float32)
  unreached_boundaries = tf.reshape(
      tf.where(tf.greater(step_boundaries, global_step)), [-1])
  unreached_boundaries = tf.concat([unreached_boundaries, [len(boundaries)]], 0)
  index = tf.reshape(tf.reduce_min(unreached_boundaries), [1])
  return tf.reshape(tf.slice(learning_rates, index, [1]), [])


global_step = tf.train.get_or_create_global_step()
lr = manual_stepping(global_step, lr_bounds, lr_vals)

# lr = tf.train.exponential_decay(learning_rate=1e-3, 
#                                 global_step=global_step, 
#                                 decay_steps=5000, 
#                                 decay_rate=0.9, 
#                                 staircase=True)
# lr = 1e-3
tf.summary.scalar('learning_rate', lr)

<tf.Tensor 'learning_rate:0' shape=() dtype=string>

In [4]:
# Graph definition.
dense = feature_batch
for i in range(len(net)):
    dense = tf.layers.dense(
        dense,
        net[i],
#         kernel_initializer=tf.truncated_normal_initializer(stddev=20.0),
#         kernel_initializer=tf.initializers.variance_scaling(scale=10.0),
        kernel_initializer=tf.initializers.variance_scaling(),
#         kernel_initializer=tf.initializers.orthogonal(),
#         kernel_initializer=tf.initializers.uniform_unit_scaling(),
#         kernel_initializer=tf.initializers.random_normal(stddev=10),
#         kernel_initializer=tf.initializers.random_uniform(-100, 100),
#         kernel_initializer=tf.initializers.zeros(),
#         bias_initializer=tf.initializers.zeros(),
#         activation=tf.nn.sigmoid,
        activation=tf.nn.relu,
#         activation=tf.nn.tanh,
    )
    if dropout_keepprob is not None:
        dense = tf.nn.dropout(dense, dropout_keepprob, seed=1234)
out = tf.layers.dense(
    dense,
    1,
#         kernel_initializer=tf.truncated_normal_initializer(stddev=20.0),
#         kernel_initializer=tf.initializers.variance_scaling(scale=10.0),
#         kernel_initializer=tf.initializers.orthogonal(),
#         kernel_initializer=tf.initializers.uniform_unit_scaling(),
#         kernel_initializer=tf.initializers.random_normal(stddev=10),
        kernel_initializer=tf.initializers.variance_scaling(),
#         kernel_initializer=tf.initializers.random_uniform(-100, 100),
#         kernel_initializer=tf.initializers.zeros(),
#         bias_initializer=tf.initializers.zeros(),
)


all_vars = tf.trainable_variables()
regularizer = tf.contrib.layers.l1_regularizer(scale=0.0)  # TODO: scale=?
# regularizer = tf.contrib.layers.l1_l2_regularizer()
regularization_penalty = tf.contrib.layers.apply_regularization(
    regularizer, all_vars)
print('out', out, 'label_batch', label_batch)
# NOTE: critical to reshape / or keep the order correct...
# out: [B, 1]
# label_batch [B]
# loss = tf.reduce_mean(tf.square(out - label_batch))
# loss = tf.reduce_mean(tf.square(out - label_batch)) 
loss = tf.reduce_mean(tf.abs(
    tf.reshape(out,[-1]) - label_batch)) + regularization_penalty

# loss = tf.reduce_mean(tf.abs(tf.reshape(out,[-1]) - label_batch))

optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_step = optimizer.minimize(
    loss, global_step=global_step)

# For visualization.
tf.summary.scalar('Train Loss', loss)
for v in all_vars:
    tf.summary.histogram(v.name, v)
all_vars_norms = [tf.norm(v) for v in all_vars]
print(len(all_vars))

Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Scale of 0 disables regularizer.
out Tensor("dense_2/BiasAdd:0", shape=(?, 1), dtype=float32) label_batch Tensor("IteratorGetNext_1:1", shape=(?,), dtype=float32)
INFO:tensorflow:Summary name Train Loss is illegal; using Train_Loss instead.
INFO:tensorflow:Summary name dense/kernel:0 is illegal; using dense/kernel_0 instead.
INFO:tensorflow:Summary name dense/bias:0 is illegal; using dense/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
6


In [5]:
# Train.
sess_args = {
             'checkpoint_dir': os.getcwd(),
             'save_checkpoint_secs': 60, 
             'save_summaries_secs': 15,
             'log_step_count_steps': 5000,
             'hooks': [tf.train.NanTensorHook(loss),
                       tf.train.StopAtStepHook(last_step=2000000),
#                       tf.train.LoggingTensorHook([global_step, loss], every_n_iter=500)
                      ],
            }

np.set_printoptions(precision=4)

def Denormalize(vec, vec_mean, vec_std):
    return vec * (vec_std + EPS) + vec_mean

loss_vals = collections.deque(maxlen=40)
i = 0
with tf.train.MonitoredTrainingSession(**sess_args) as sess:
    sess.run(iterator_init)
    while not sess.should_stop():
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     running_test_loss = 0.0
#     while True:
        _, loss_val, global_step_val, true_feat_train, true_labels_train, pred_labels_train,\
           feat_mean_val, feat_std_val, lab_mean_val, lab_std_val = sess.run(
            [train_step, loss, global_step, feature_batch, label_batch, out, 
             feat_mean, feat_std, lab_mean, lab_std])

        if i % 1000 == 0:
#             sess.run(test_iterator_init)
#             test_loss_val, actual_labels, predicted_labels = sess.run([loss, label_batch, out])

#             loss_vals.append(test_loss_val)
#             avg_test_loss = np.mean(loss_vals)
#             print('global_step', global_step_val, ', loss', loss_val, ', avg test loss', avg_test_loss)
            print('global_step', global_step_val, ', loss', loss_val)
            if i % 10000 == 0:
#                 print(' true labels', actual_labels[:20].reshape(-1,))
#                 print(' pred labels', predicted_labels[:20].reshape(-1,))
                print(' true feats train', true_feat_train[:1])
                print(' true labels train', Denormalize(true_labels_train[:10].reshape(-1,), lab_mean_val, lab_std_val))
                print(' pred labels train', Denormalize(pred_labels_train[:10].reshape(-1,), lab_mean_val, lab_std_val))
                print(' all vars norms', sess.run(all_vars_norms))
                print(' label mean', lab_mean_val, 'std', lab_std_val)
        i += 1
#         if i >= 10000:
#             break

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /Users/zongheng/Dropbox/workspace/riselab/rlqopt/expr_learning/model.ckpt.
global_step 1 , loss 0.8389878
 true feats train [[ 0.      0.      0.      0.      0.     -0.2526 -0.2015 -0.2001 -0.2366
  -0.1803 -0.2183 -0.2068 -0.227  -0.2015 -0.2538 -0.2669 -0.2593 -0.1919
  -0.1987 -0.2081 -0.1697 -0.1932 -0.1743 -0.1833  5.5001 -0.1946 -0.1904
  -0.1919 -0.1862 -0.4898  1.9814 -0.493  -0.5101 -0.5023 -0.7533]]
 true labels train [  1.   1.   1. 632.  84.   9.   1.   5.   1.  11.]
 pred labels train [328.2047 112.176  274.6404  71.6607 206.6745 162.6405 233.6257 246.828
 205.6633 179.6972]
 all vars norms [14.236575, 0.015989471, 14.045793, 0.015992627, 0.9137814, 0.0009999989]
 label mean 260.3535 std 341.22647
global_step 1001 , loss 0.052482788
global_step 2001 , loss 0.0333187

KeyboardInterrupt: 